In [1]:
x, y, z = var('x y z')
#f = x^3 - (y^2)*z + z^3
#P = [2, 3, 1]
#Q = [0, 1, 1]
#O = [0, 1, 0]
f = x^3 - (y^2)*z - 2*z^3
P = [3, 5, 1]
O = [0, 1, 0]

def point_normal_form(P):
    '''
    Parameters
    ----------
    P : list
        point with a component equal to 1.

    Returns
    -------
    P : list
        the same point in projective space, with integer coprime components.
    '''
    fractional = False
    for el in P:
        if el.denominator() != 1:
            fractional = True
            break
    
    if fractional:
        dens = [el.denominator() for el in P]
        lcm = 1
        for d in dens:
            lcm = int(lcm*d/gcd(lcm, d))
        P = [lcm*el for el in P]
    
    return P


def is_equal_points(P, Q):
    '''
    Parameters
    ----------
    P : list
        first point.
    Q : list
        second point.

    Returns
    -------
    val : Bool
        True if P = Q in projective space, False otherwise.
    '''
    for i in P:
        if i != 0:
            break
    
    j = Q[P.index(i)]
    if j == 0:
        val = False
    else:
        P = [P[k]/i for k in range(len(P))]
        Q = [Q[k]/j for k in range(len(Q))]
        if P == Q:
            val = True
        else:
            val = False
            
    return val

def is_equal_lines(l1, l2):
    '''
    Parameters
    ----------
    l1 : expr
        first line.
    l2 : expr
        second line.

    Returns
    -------
    val : Bool
        True if the lines given by l1 = 0 and l2 = 0 are the same, False otherwise
    '''
    P1 = list(map(lambda w: w.subs(x=1,y=1,z=1),l1.operands()))
    P2 = list(map(lambda w: w.subs(x=1,y=1,z=1),l2.operands()))
    return is_equal_points(P1, P2)

def tangent_line(f, P):
    '''
    Parameters
    ----------
    f : expr
        cubic curve.
    P : list
        point.

    Returns
    -------
    l : expr
        tangent line at P.
    '''
    A = diff(f, x).subs(x == P[0], y == P[1], z == P[2])
    B = diff(f, y).subs(x == P[0], y == P[1], z == P[2])
    C = diff(f, z).subs(x == P[0], y == P[1], z == P[2])
    l = A*x + B*y + C*z
    
    return l

def line_containing(P, Q):
    '''
    Parameters
    ----------
    P : list
        first point.
    Q : list
        second point.

    Returns
    -------
    l : expr
        line containing P and Q.
    '''
    a, b, c = var('a b c')
    l = a*x + b*y + c*z
    exprs = [l.subs(x == point[0], y == point[1], z == point[2]) for point in [P, Q]]
    sols1 = solve([exprs[0] == 0, exprs[1] == 0], a, b, c)[0]
    l = l.subs(sols1[0], sols1[1], sols1[2])
    new_var = l.variables()[0]
    l = l.subs(new_var == 1)
    return l

def intersection(l, f):
    '''
    Parameters
    ----------
    l : expr
        line.
    f : expr
        cubic curve.

    Returns
    -------
    Rs : list
        list of intersection points of the line and the curve.
    '''
    sols2 = solve([f == 0, l == 0], x, y, z, multiplicities=True)
    Rs = []
    for i in sols2:
        for j in i:
            # check it's not of the form x == 0 or y == 0
            if len(j.variables()) > 1:
                new_var = j.variables()[0]
                i = [i[k].subs(new_var == 1) for k in range(len(i))]
                R = [i[0].rhs(), i[1].rhs(), i[2].rhs()]
                Rs.append(R)
                break   
    
    return Rs

def cubic_add(f, P, Q, O):
    '''
    Parameters
    ----------
    f : expr
        cubic curve.
    P : list
        first point.
    Q : list
        second point.
    O : list
        zero of the abelian group.

    Returns
    -------
    S : list
        P + Q.
    '''
    # find line l containing P, Q
    was_tangent = False
    
    if is_equal_points(P, Q):
        was_tangent = True
        l1 = tangent_line(f, P)
    else:
        l1 = line_containing(P, Q)
    
    # find intersection R of line l and curve C
    Rs = intersection(l1, f)
    # case where a point is repeated in the intersection
    if len(Rs) == 2:
        if was_tangent:
            for R in Rs:
                if not is_equal_points(R, P):
                    break
        else:
            for R in Rs:
                if is_equal_lines(l1, tangent_line(f, R)):
                    break
    
    else:
        for R in Rs:
            if (not is_equal_points(R, P)) and (not is_equal_points(R, Q)):
                break
    
    # find line l' containing R and O
    was_tangent = False
    if is_equal_points(R, O):
        was_tangent = True
        l2 = tangent_line(f, R)
    else:
        l2 = line_containing(R, O)
    
    # find intersection S of line l' and curve C
    Ss = intersection(l2, f)
    if len(Ss) == 2:
        if was_tangent:
            for S in Ss:
                if not is_equal_points(S, R):
                    break
        
        else:
            for S in Ss:
                if is_equal_lines(l2, tangent_line(f, S)):
                    break
    else:
        for S in Ss:
            if (not is_equal_points(S, R)) and (not is_equal_points(S, O)):
                break

    S = point_normal_form(S)
    
    return S

#print(cubic_add(f, [2,3,1], [2,3,1], O))

def cubic_n_times(f, P, O, n):
    '''
    Parameters
    ----------
    f : expr
        cubic curve.
    P : list
        point.
    O : list
        zero of the abelian group.
    n : int
        number of times to add P to itself

    Returns
    -------
    S : list
        n*P.
    '''
    S = P
    while n > 1:
        S = cubic_add(f, S, P, O)
        n -= 1

    return S

#print(cubic_n_times(f, P, O, 5))
N = cubic_n_times(f, P, O, 2)
print(N)


[1290, -383, 1000]
